# Mach-Zehnder Interferometer as Hadamard gate
This notebook uses qiskit to describe the outcome of the Single Photon Mach-Zehnder experiment as a Hadamard gate and a measurement.
It serves as a simple example how to describe Quantum Mechanical experiments as a circuit of quantum gates.

### Qiskit
We are using Qiskit to describe, draw and simulate the quantum circuit.
To include the visualization add the tag in the square brackets.
Note: in zsh this has to be encapsulated in single quotes.

In [ ]:
!pip3 install 'qiskit[visualization]'

### Libraries
For the first simulation we use the QasmSimulator:

> "The QasmSimulator backend is designed to mimic an actual device. It executes a Qiskit QuantumCircuit and returns a count dictionary containing the final values of any classical registers in the circuit. 
The circuit may contain gates measure, reset, conditionals, and other advanced simulator options." (https://sooluthomas.github.io/testTranslation/aer/simulate_experiments.html)

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

### Mach-Zehnder interferometer
(https://en.wikipedia.org/wiki/Mach–Zehnder_interferometer)
![MZ](https://upload.wikimedia.org/wikipedia/commons/thumb/c/c9/Mach-Zehnder_interferometer.svg/640px-Mach-Zehnder_interferometer.svg.png)

We can model a photon going through the interferometer by assigning a probability amplitude to each of the two possible paths: 

* the "lower" path which starts from the left, goes straight through both beam splitters, and ends at the top 
* the "upper" path which starts from the bottom, goes straight through both beam splitters, and ends at the right.

The quantum state describing the photon is therefore a vector $\psi \in \mathbb{C}^2$ that is a superposition of the "lower" path 
$$
\psi_l = \begin{pmatrix} 1 \\
0 
\end{pmatrix}
$$

and the "upper" path 

$$
\psi_u = \begin{pmatrix} 0 \\
1 
\end{pmatrix}
$$

that is, $\psi = \alpha \psi_l + \beta \psi_u$ for complex $\alpha,\beta$ such that $|\alpha|^2+|\beta|^2 = 1$.

Both beam splitters are modelled as the unitary matrix 

$$
B = \frac1{\sqrt2}
\begin{pmatrix} 
    1 & i \\ 
    i & 1 
\end{pmatrix}
$$

which means that when a photon meets the beam splitter it will either stay on the same path with a probability amplitude of $1/\sqrt{2}$, or be reflected to the other path with a probability amplitude of $i/\sqrt{2}$.
The phase shifter on the upper arm is modelled as the unitary matrix 

$$
P = 
\begin{pmatrix} 
    1 & 0 \\
    0 & e^{i\Delta\Phi} 
\end{pmatrix}
$$

which means that if the photon is on the "upper" path it will gain a relative phase of $\Delta\Phi$, and it will stay unchanged if it is on the lower path.

A photon that enters the interferometer from the left will then end up described by the state:

$$
BPB\psi_l = ie^{i\Delta\Phi/2} 
\begin{pmatrix} 
    -\sin(\Delta\Phi/2) \\ 
    \cos(\Delta\Phi/2) 
\end{pmatrix}
$$

and the probabilities that it will be detected at the right or at the top are given respectively by:

$$ 
p(u) = |\langle \psi_u| BPB|\psi_l \rangle|^2 = \cos^2 \frac{\Delta \Phi}{2}\\
p(l) = |\langle \psi_l| BPB|\psi_l \rangle|^2 = \sin^2 \frac{\Delta \Phi}{2}
$$

One can therefore use the Mach–Zehnder interferometer to estimate the Phase by estimating these probabilities.

Let's now assume as in the double slit example we know which path the photon takes (eg blocking one path).
In this case there will no longer be interference between the paths, and the probabilities are given by $p(u)=p(l) = 1/2$, independently of the phase $\Delta\Phi$.

## Hadamard Gate - Model
For our model circuit we can now assume that the photon is in a superposition of the top and bottom path.
The quamtum logical gate that constructs a superposition is called the Hadamard gate: (https://en.wikipedia.org/wiki/Quantum_logic_gate#Hadamard_(H)_gate)

![Hadamard matrix](https://upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Hadamard_gate.svg/320px-Hadamard_gate.svg.png)

which can act on a single qubit (in our case the Mach-Zehnder Interferometer).

It maps the basis state $|0\rangle$ to $\frac{|0\rangle + |1\rangle}{\sqrt{2}}$ and  $|1\rangle$ to $\frac{|0\rangle - |1\rangle}{\sqrt{2}}$, which means that a measurement will have equal probabilities to result in 1 or 0 (i.e. creates a superposition).
It represents a rotation of $\pi$ about the axis $(\hat{x}+\hat{z})/\sqrt{2}$ at the Bloch sphere (see plotting the state vector below). 
Equivalently, it is the combination of two rotations,  $\pi$ about the Z-axis, then by $\pi/2$ about the Y-axis: 

$$ 
R_y(\pi/2)R_z(\pi) = iH
$$

It is represented by the Hadamard matrix:

$$H = \frac{1}{\sqrt{2}} 
\begin{bmatrix} 
1 & 1 \\
1 & -1 
\end{bmatrix}
$$

Since $HH^\dagger=H^{\dagger}H=I$ where $I$ is the identity matrix, $H$ is a unitary matrix (like all other quantum logical gates).

### Create Quantum circuit

In [ ]:
# Create a Quantum Circuit acting on the q register
circuit = QuantumCircuit(QuantumRegister(1), ClassicalRegister(1))

# Add a H gate on qubit 0
circuit.h(0)
#circuit.z(0)
#circuit.h(0)

# Map the quantum measurement to the classical bits
circuit.measure([0], [0])

# Draw the circuit
circuit.draw()

### Simulate the quantum circuit

In [ ]:
# Use Aer's qasm_simulator
simulator = QasmSimulator()

# transpile the circuit down to low-level QASM instructions
# supported by the backend (not needed for simple circuits)
# https://en.wikipedia.org/wiki/Source-to-source_compiler
compiled_circuit = transpile(circuit, simulator)

In [ ]:
# print low level code: open Quantum Assembler
print(compiled_circuit.qasm())

In [ ]:
# Execute the circuit on the qasm simulator
job = simulator.run(compiled_circuit, shots=1000)

# Grab results from the job
result = job.result()

# Returns counts
counts = result.get_counts(circuit)
print("\nTotal count for 00 and 11 are:",counts)

In [ ]:
# Plot a histogram
plot_histogram(counts)

## Plot the state vector
To plot the state vector we do not need to (should not) do a measurement.
You can use the QasmSimulator backend and the `SaveStateVector` function, or use another simulator made for this purpose:

'statevector_simulator'

In [ ]:
from qiskit import BasicAer, execute
from qiskit.visualization import plot_state_city, plot_bloch_multivector
from qiskit.visualization import plot_state_paulivec, plot_state_hinton
from qiskit.visualization import plot_state_qsphere

In [ ]:
circuit.draw()

Let's redo our cricuit and spllit it in circuit and measurement

In [ ]:
# Create a Quantum Circuit acting on the q register
qr = QuantumRegister(1)
qcirc = QuantumCircuit(qr)

# Add a H gate on qubit 0
qcirc.h(0)
qcirc.draw()

In [ ]:
# execute the quantum circuit
backend = BasicAer.get_backend('statevector_simulator') # the device to run on
result = execute(qcirc, backend).result()
psi  = result.get_statevector(qcirc)

In [ ]:
plot_state_city(psi)

In [ ]:
plot_bloch_multivector(psi)

## Combining Quantum Circuits

In [ ]:
# measurement circuit
meas = QuantumCircuit(qr,ClassicalRegister(1))
meas.measure(qr,[0])
# Draw the circuit
meas.draw()

In [ ]:
circ = qcirc+meas
circ.draw()

In [ ]:
result = execute(circ, backend, shots=1000).result()
counts  = result.get_counts(circ)
print(counts)

In [ ]:
plot_histogram(counts)